In [1]:
!pip install tslearn
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans,silhouette_score
from tslearn.preprocessing import TimeSeriesScalerMinMax
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

!pip install pyclustering



     -------------------------------------- 460.4/460.4 kB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyclustering: filename=pyclustering-0.10.1.2-py3-none-any.whl size=2395106 sha256=173e7361f8a7102e0182dbb41c1581adf3e4cef69d416331319f33a0665e854d
  Stored in directory: c:\users\daich\appdata\local\pip\cache\wheels\e0\56\c2\abb6866a3fcd8a55862f1df8a18f57805c3a78fed9a9023cb9
Successfully built pyclustering


In [2]:
from sklearn import datasets, preprocessing
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import glob
import re

from pyclustering.cluster import gmeans, xmeans
import itertools
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
import platform

In [3]:
file = 'check_monsakun_log_02.csv'
df = pd.read_csv(file,index_col=0)
df=df.dropna(0)
df2 = df.mean()
# display(df2)
df2.to_csv('check_monsakun_mean_02.csv')

C:\Users\daich\AppData\Local\Temp\ipykernel_22620\4056114110.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  df=df.dropna(0)


In [12]:
#　物語戦略
files1= glob.glob('monsakun_log_??.csv')
files2= glob.glob('problems_shobi2017-?.csv')
for num1,file1 in enumerate(files1):
  fnum1 = re.sub(r"\D","",file1)
  print("fnum1=",fnum1)
  fnum2 = fnum1[-1]
  
  
  file2 = 'problems_shobi2017-'+str(fnum2)+'.csv'
  print(file1,file2)
  df1 = pd.read_csv(file1)
  df2 = pd.read_csv(file2)

  df3 = pd.merge(df1,df2,on=['lv','asg','card'],how='left')
  df3['story_st']=0
  df3['relation_st']=0
  # df.loc[(df['stp']==1)&(df['card']==3),'relation_st']=1
  df3.loc[(df3['stp']==1)&(df3['type0']=='r1'),'relation_st']=1
  df3.loc[(df3['stp']==1)&(df3['type0']=='r0'),'relation_st']=1

  for index,row in df3.iterrows():
    if row['stp']==1:
      relation_st = row['relation_st']
    elif row['ope1']!='PROB_BEGIN':
      df3.loc[index,'relation_st']=relation_st

  set_ct = 0
  for index,row in df3.iterrows():
    # print(row['session'],session,row['ope1'],row['check'],row['type2'])
    if row['ope1']=='PROB_BEGIN':
      session = row['session']
      set_ct = 0
      story1 = 0
      story2 = 0
    if row['session'] == session and row['ope1']=='SET' and row['check']==0:
      # print(index,set_ct,row['type2'],row['slot'])
      set_ct += 1
      # print(set_ct)
      if set_ct == 1 and row['type2'] =='existence' and row['slot']==1:
        story1 += 1
        story2 += 1
      elif set_ct == 2 and row['type2'] in ['increase','decrease'] and row['slot'] == 2:
        story1 += 1
      elif set_ct == 2 and row['type2'] =='existence' and row['slot'] == 2:
        story2 += 1
      elif set_ct == 3 and (row['type2'] in ['combination','comp_more']) and row['slot']==3:
        story2 +=1
      elif set_ct == 3 and (row['type2'] =='existence') and row['slot']==3:
        story1 +=1 
    if row['session'] == session and row['ope1']=='CHECK':
      if story2 ==3 or story1 == 3:
        # for index,row in df3.iterrows():
        #   if session == row['session']:
        #     df3.loc[index,'story_st']=1
        df3.loc[index,'story_st']=1
    # if index == 100:
    #   break
  df3.to_csv('monsakun_log_with_type/monsakun_log_'+str(fnum1)+'_with_type.csv')
  df3['q']=df3['lv'].astype(str).str.cat(df3['asg'].astype(str),sep='_')
  df4 = df3.loc[df3['ope1']=='CHECK']
  df4.to_csv('monsakun_log_check_r_s/monsakun_log_check_r_s'+str(fnum1)+'.csv')
  pd.crosstab([df4['relation_st'],df4['story_st']],[df4['q'],df4['jdg']]).to_csv('r_s_first_check/r_s_first_check_'+str(fnum1)+'.csv')

fnum1= 02
monsakun_log_02.csv problems_shobi2017-2.csv
fnum1= 03
monsakun_log_03.csv problems_shobi2017-3.csv
fnum1= 04
monsakun_log_04.csv problems_shobi2017-4.csv
fnum1= 05
monsakun_log_05.csv problems_shobi2017-5.csv


In [ ]:
# pd.crosstab([df3['relation_st'],df3['story_st']],[df3['q'],df3['jdg']]).to_csv('r_s_first_check_'+'02')+'.csv')
#新しい列の作成（レベル＿問題番号）
df3['q']=df3['lv'].astype(str).str.cat(df3['asg'].astype(str),sep='_')
df3.loc[df['ope1']=='CHECK'].to_csv('monsakun_log_check_r_s'+str(fnum)+'.csv')
pd.crosstab([df3['relation_st'],df3['story_st']],[df3['q'],df3['jdg']]).to_csv('r_s_first_check_'+'02'+'.csv')


In [ ]:
}files= glob.glob('monsakun_log_??_check.csv')
for num,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)

  df = pd.read_csv(file)
  
  display(df[['lv','check']].groupby('lv').mean())
  # print(df['lv'])

fnum= 04


,check
lv,
1,1.146179
2,1.250000
3,15.467490


fnum= 02


,check
lv,
1,1.149123
2,1.113208
3,2.633898


fnum= 05


,check
lv,
1,1.180000
2,1.593272
3,1.635057


fnum= 03


,check
lv,
1,1.282158
2,1.262097
3,7.938586


In [ ]:
files= glob.glob('monsakun_log_??.csv')
for num,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)

  df = pd.read_csv(file)
  df['relation_st']=0
  df['story_st']=0
  # display(df)

#関係文の定義
  # if fnum == '02' or fnum == '05':
  #   df.loc[(df['stp']==1)&(df['card']==3),'relation_st']='card_3'
  #   df.loc[(df['stp']==1)&(df['card']==5),'relation_st']='card_5'
  # else:
  #   df.loc[(df['stp']==1)&(df['card']==2),'relation_st']='card_2'
  #   df.loc[(df['stp']==1)&(df['card']==5),'relation_st']='card_5'
  if fnum == '02' or fnum == '05':
    df.loc[(df['stp']==1)&(df['card']==3),'relation_st']=1
    df.loc[(df['stp']==1)&(df['card']==5),'relation_st']=1
  else:
    df.loc[(df['stp']==1)&(df['card']==2),'relation_st']=1
    df.loc[(df['stp']==1)&(df['card']==5),'relation_st']=1

  for index, row in df.iterrows():
    
    if row['ope1']=='CHECK':
      check = 1
    else:
      check = 0
    if row['ope1']=='PROB_BEGIN':
      begin_session = row['session']
      set1 = 0
      set2 = 0
      set3 = 0
      cond1 = 0
      cond2 = 0
      set_ct = 0
    # print('set_ct',set_ct)
    elif row['ope1']=='SET':
      set_ct += 1
      if fnum == '02' or fnum =='05':
        if set_ct == 1 and row['slot1'] not in [0,3,5] and row['slot2'] == 0\
          and row['slot3'] == 0:
          cond1 += 1
          cond2 += 1
        elif set_ct == 2 and row['slot1'] not in [0,3,5] and row['slot2'] not in [0,3,5]\
              and row['slot3'] == 0:
          cond1 += 1
        elif set_ct == 2 and row['slot1'] not in [0,3,5] and row['slot2'] == 5 and \
              row['slot3'] == 0:
          cond2 += 1
        elif set_ct == 3 and row['slot1'] not in [0,3,5] and row['slot2'] ==5 and\
              row['slot3'] not in [0,3,5]:
          cond2 += 1
        elif set_ct ==3 and row['slot1'] not in [0,3,5] and row['slot2'] not in [0,3,5]\
              and row['slot3'] ==3:
          cond1 +=1
        # elif row['slot1'] not in [0,3,5]:
        #   if row['slot2']==5:
        #     if row['slot3'] not in [0,3,5]:
        #       df.loc[index,'story_st']=1
      else:
        if set_ct == 1 and row['slot1'] not in [0,2,5] and row['slot2'] == 0\
          and row['slot3'] == 0:
          cond1 += 1
          cond2 += 1
        elif set_ct == 2 and row['slot1'] not in [0,2,5] and row['slot2'] not in [0,2,5]\
              and row['slot3'] == 0:
          cond1 += 1
        elif set_ct == 2 and row['slot1'] not in [0,2,5] and row['slot2'] == 2 and \
              row['slot3'] == 0:
          cond2 += 1
        elif set_ct == 3 and row['slot1'] not in [0,2,5] and row['slot2'] ==2 and\
              row['slot3'] not in [0,2,5]:
          cond2 += 1
        elif set_ct ==3 and row['slot1'] not in [0,2,5] and row['slot2'] not in [0,2,5]\
              and row['slot3'] ==5:
          cond2 += 1
    elif row['ope1']=='CHECK':
      if cond1 == 3 or cond2 ==3:
        df.loc[index,'story_st']=1

    if row['relation_st']!=0:
      relation_st = row['relation_st']
      session = row['session']
      # print(relation_st)  
      # print(row['session'],session)
    if (row['session']==session):
      if(row['stp']!=1):
        df.loc[index,'relation_st']=relation_st
  df.to_csv('monsakun_log_relation_story_'+str(fnum)+'.csv')
  df.loc[df['ope1']=='CHECK'].to_csv('monsakun_log_check_relation_story_'+str(fnum)+'.csv')

fnum= 03
fnum= 05
fnum= 02
fnum= 04


In [ ]:
# 関係文戦略と物語戦略の比較
files = glob.glob('monsakun_log_check_relation_story_??.csv')
for index,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)

  df = pd.read_csv(file,index_col=0)
  # df['relation_st'] = 0

  # #関係文の定義
  # if fnum == '02' or fnum == '05':
  #   df.loc[(df['stp']==1)&(df['card']==3),'relation_st']='card_3'
  #   df.loc[(df['stp']==1)&(df['card']==5),'relation_st']='card_5'
  # else:
  #   df.loc[(df['stp']==1)&(df['card']==2),'relation_st']='card_2'
  #   df.loc[(df['stp']==1)&(df['card']==5),'relation_st']='card_5'

  #新しい列の作成（レベル＿問題番号）
  df['q']=df['lv'].astype(str).str.cat(df['asg'].astype(str),sep='_')

  #チェック数１での結果(関係文と物語)
  df2 = df[df['check']==1]
  df2.to_csv('check_1_'+str(fnum)+'.csv')
  pd.crosstab(df2['relation_st'],[df2['q'],df2['jdg']]).to_csv('relation_first_check_'+str(fnum)+'.csv')
  pd.crosstab([df2['relation_st'],df2['story_st']],[df2['q'],df2['jdg']]).to_csv('relation_story_first_check_'+str(fnum)+'.csv')
  pd.pivot_table(df2,index='InputID',columns=['q'],values=['relation_st','story_st']).to_csv('InputID_q_relation_story'+str(fnum)+'.csv')
  pd.crosstab(df2['InputID'],[df2['q']]).to_csv('InputID_q_relation_story'+str(fnum)+'.csv')
  pd.crosstab(df2['relation_st'],df2['q'],margins=True).to_csv('relation_st_per_q_'+str(fnum)+'.csv')

  #戦略ごとのチェック数
  df2 = df[df['jdg']=='s']
  # if index == 0:
  df2.to_csv('relation_st'+str(fnum)+'.csv')


  pd.pivot_table(df2,index='relation_st',columns=['q'],values='check',aggfunc=[np.mean,len]).to_csv('strategy_check_'+str(fnum)+'.csv')
  pd.pivot_table(df2,index='relation_st',columns=['q'],values='stp',aggfunc=[np.mean,len]).to_csv('strategy_step_'+str(fnum)+'.csv')

fnum= 02
fnum= 03
fnum= 05
fnum= 04


In [ ]:
files = glob.glob('monsakun_log_check_relation_story_02.csv')

for index,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)

  df = pd.read_csv(file,index_col=0)
  df = df[df['check']==1]
  df.to_csv('02.csv')
  df = df.replace({'relation_st':{1:'relation_st'},'story_st':{1:'story_st'}})
  display(df)
  df['strategy']=df['relation_st'].astype(str).str.cat(df['story_st'].astype(str))
  df['q'] = df['lv'].astype(str).str.cat(df['asg'].astype(str),sep='_')
  df = df.replace({'strategy':{'0*':''}},regex=True)
  df = df.replace({'strategy':{'':'0'}},regex=True)
  # print(df)
  pd.crosstab(df['InputID'],[df['q'],df['strategy'],df['jdg']]).to_csv('InputID_relation_story_'+str(fnum)+'.csv')
  # display(df)

fnum= 02


,date,time,ans,InputID,lv,asg,stp,ope1,slot,card,...,jdg,C.S,ST.S,N.S,SS.S,O.S,validity,trial,relation_st,story_st
4,2017/10/09,14:50:55,521,31710071,1,1,3,CHECK,0,0,...,f,0,0,1,1,1,3,1,0,0
14,2017/10/09,14:51:46,321,31710071,1,2,3,CHECK,0,0,...,s,1,1,1,1,1,5,1,0,story_st
19,2017/10/09,14:52:10,321,31710071,1,3,3,CHECK,0,0,...,s,1,1,1,1,1,5,1,0,story_st
24,2017/10/09,14:52:44,321,31710071,1,4,3,CHECK,0,0,...,s,1,1,1,1,1,5,1,relation_st,0
29,2017/10/09,14:53:09,521,31710071,1,5,3,CHECK,0,0,...,f,0,0,1,1,1,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,2017/10/09,15:07:25,213,31710074,3,1,7,CHECK,0,0,...,s,1,1,1,1,1,5,1,relation_st,0
4184,2017/10/09,15:07:39,213,31710074,3,2,3,CHECK,0,0,...,s,1,1,1,1,1,5,1,relation_st,0
4189,2017/10/09,15:07:50,213,31710074,3,3,3,CHECK,0,0,...,s,1,1,1,1,1,5,1,relation_st,0
4196,2017/10/09,15:08:16,213,31710074,3,4,5,CHECK,0,0,...,s,1,1,1,1,1,5,1,relation_st,0


In [ ]:

files = glob.glob('monsakun_log_check_relation_st_??.csv')
for index,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)

  df = pd.read_csv(file,index_col=0)
  # df['relation_st'] = 0

  # #関係文の定義
  # if fnum == '02' or fnum == '05':
  #   df.loc[(df['stp']==1)&(df['card']==3),'relation_st']='card_3'
  #   df.loc[(df['stp']==1)&(df['card']==5),'relation_st']='card_5'
  # else:
  #   df.loc[(df['stp']==1)&(df['card']==2),'relation_st']='card_2'
  #   df.loc[(df['stp']==1)&(df['card']==5),'relation_st']='card_5'

  #新しい列の作成（レベル＿問題番号）
  df['q']=df['lv'].astype(str).str.cat(df['asg'].astype(str),sep='_')

  #チェック数１での結果
  df2 = df[df['check']==1]
  df2.to_csv('check_1_'+str(fnum)+'.csv')
  pd.crosstab(df2['relation_st'],[df2['q'],df2['jdg']]).to_csv('relation_first_check_'+str(fnum)+'.csv')
  pd.crosstab([df2['relation_st'],df2['story_st']],[df2['q'],df2['jdg']]).to_csv('relation_story_first_check_'+str(fnum)+'.csv')
  pd.pivot_table(df2,index='InputID',columns=['q'],values=['relation_st']).to_csv('InputID_q_'+str(fnum)+'.csv')
  pd.crosstab(df2['relation_st'],df2['q'],margins=True).to_csv('relation_st_per_q_'+str(fnum)+'.csv')

  #戦略ごとのチェック数
  df2 = df[df['jdg']=='s']
  # if index == 0:
  df2.to_csv('relation_st'+str(fnum)+'.csv')


  pd.pivot_table(df2,index='relation_st',columns=['q'],values='check',aggfunc=[np.mean,len]).to_csv('strategy_check_'+str(fnum)+'.csv')
  pd.pivot_table(df2,index='relation_st',columns=['q'],values='stp',aggfunc=[np.mean,len]).to_csv('strategy_step_'+str(fnum)+'.csv')

fnum= 04
fnum= 05
fnum= 03
fnum= 02


In [ ]:
files = glob.glob('relation_st??.csv')
usr = {}
for index,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)

  df = pd.read_csv(file,index_col=0)

  usr[fnum]=(len(df['InputID'].unique()))
  # usr[fnum]=(df['check'].mean())

usr = sorted(usr.items())
print(usr)


fnum= 05
fnum= 03
fnum= 04
fnum= 02
[('02', 40), ('03', 42), ('04', 56), ('05', 53)]


In [ ]:
files = glob.glob('relation_first_check_??.csv')
for index,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)
  df=pd.read_csv(file,index_col=0,header=None).T
  df['lv']=df['q'].str[0]
  df['0']=df['0'].astype(float)
  df['1']=df['1'].astype(float)
  df=df.reindex(columns=['lv','jdg','q','0','1'])
  # print(df.dtypes)
  # print(df.groupby(['lv','jdg']).get_group(('1','f')))
  print(df)
  print(df['lv'].value_counts(normalize=True))
  df[['jdg','0','1','lv']].groupby(['lv','jdg']).mean().T.to_csv('relation_first_check_mean_'+str(fnum)+'.csv')

fnum= 04
0  lv jdg    q     0     1
1   1   f  1_1   5.0   1.0
2   1   s  1_1  42.0   6.0
3   1   f  1_2   5.0   3.0
4   1   s  1_2  44.0   2.0
5   1   f  1_3   3.0   1.0
6   1   s  1_3  46.0   4.0
7   1   f  1_4   2.0   0.0
8   1   s  1_4  43.0   9.0
9   1   f  1_5   1.0   0.0
10  1   s  1_5  49.0   4.0
11  2   f  2_1   6.0   3.0
12  2   s  2_1  39.0   6.0
13  2   f  2_2   5.0   0.0
14  2   s  2_2  45.0   4.0
15  2   f  2_3   5.0   3.0
16  2   s  2_3  43.0   3.0
17  2   f  2_4   3.0   1.0
18  2   s  2_4  45.0   5.0
19  2   f  2_5   6.0   0.0
20  2   s  2_5  44.0   4.0
21  3   f  3_1  38.0   5.0
22  3   s  3_1   5.0   3.0
23  3   f  3_2  14.0   5.0
24  3   s  3_2  27.0   5.0
25  3   f  3_3  11.0   8.0
26  3   s  3_3  23.0   9.0
27  3   f  3_4   8.0   1.0
28  3   s  3_4  30.0  12.0
29  3   f  3_5   6.0   2.0
30  3   s  3_5  34.0   9.0
1    0.333333
2    0.333333
3    0.333333
Name: lv, dtype: float64
fnum= 02
0  lv jdg    q     0     1
1   1   f  1_1   8.0   1.0
2   1   s  1_1  25.0   6

In [ ]:
files = glob.glob('relation_first_check_??.csv')
for index,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)
  df=pd.read_csv(file,index_col=0,header=None).T
  df['lv']=df['q'].str[0]
  df['0']=df['0'].astype(float)
  df['1']=df['1'].astype(float)
  df=df.reindex(columns=['lv','jdg','q','0','1'])
  # print(df.dtypes)
  # print(df.groupby(['lv','jdg']).get_group(('1','f')))
  df['lv']=df['lv']+'_'+df['jdg']
  print(df)
  print(df['lv'].value_counts(normalize=True))
  df[['jdg','0','1','lv']].groupby(['lv',]).sum().T.to_csv('relation_first_check_ratio_'+str(fnum)+'.csv')

fnum= 04
0    lv jdg    q     0     1
1   1_f   f  1_1   5.0   1.0
2   1_s   s  1_1  42.0   6.0
3   1_f   f  1_2   5.0   3.0
4   1_s   s  1_2  44.0   2.0
5   1_f   f  1_3   3.0   1.0
6   1_s   s  1_3  46.0   4.0
7   1_f   f  1_4   2.0   0.0
8   1_s   s  1_4  43.0   9.0
9   1_f   f  1_5   1.0   0.0
10  1_s   s  1_5  49.0   4.0
11  2_f   f  2_1   6.0   3.0
12  2_s   s  2_1  39.0   6.0
13  2_f   f  2_2   5.0   0.0
14  2_s   s  2_2  45.0   4.0
15  2_f   f  2_3   5.0   3.0
16  2_s   s  2_3  43.0   3.0
17  2_f   f  2_4   3.0   1.0
18  2_s   s  2_4  45.0   5.0
19  2_f   f  2_5   6.0   0.0
20  2_s   s  2_5  44.0   4.0
21  3_f   f  3_1  38.0   5.0
22  3_s   s  3_1   5.0   3.0
23  3_f   f  3_2  14.0   5.0
24  3_s   s  3_2  27.0   5.0
25  3_f   f  3_3  11.0   8.0
26  3_s   s  3_3  23.0   9.0
27  3_f   f  3_4   8.0   1.0
28  3_s   s  3_4  30.0  12.0
29  3_f   f  3_5   6.0   2.0
30  3_s   s  3_5  34.0   9.0
1_f    0.166667
1_s    0.166667
2_f    0.166667
2_s    0.166667
3_f    0.166667
3_s    0.166

In [ ]:
!zip -r relation_strategy.zip /content

from google.colab import files

files.download("relation_strategy.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2022.11.09/ (stored 0%)
  adding: content/.config/logs/2022.11.09/00.02.34.309262.log (deflated 54%)
  adding: content/.config/logs/2022.11.09/00.01.25.380598.log (deflated 91%)
  adding: content/.config/logs/2022.11.09/00.02.59.524518.log (deflated 55%)
  adding: content/.config/logs/2022.11.09/00.03.00.357803.log (deflated 54%)
  adding: content/.config/logs/2022.11.09/00.01.54.934811.log (deflated 53%)
  adding: content/.config/logs/2022.11.09/00.02.19.125189.log (deflated 86%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: co

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files = glob.glob('monsakun_log_*_check.csv')
for index,file in enumerate(files):
  fnum = re.sub(r"\D","",file)
  fnum = fnum[:2]
  print("fnum=",fnum)

  fname = 'monsakun_log_02_check.csv'
  df = pd.read_csv(file,index_col=0)
  df['relation_st'] = 0

  #関係文の定義
  if fnum == '02' or fnum == '05':
    df.loc[df['slot1']==3,'relation_st'] = 'card_3'
    df.loc[df['slot1']==5,'relation_st'] = 'card_5'
  else:
    df.loc[df['slot1']==2,'relation_st'] = 'card_2'
    df.loc[df['slot1']==5,'relation_st'] = 'card_5'

  #新しい列の作成（レベル＿問題番号）
  df['q']=df['lv'].astype(str).str.cat(df['asg'].astype(str),sep='_')

  #チェック数１での結果
  df2 = df[df['check']==1]

  pd.crosstab(df2['relation_st'],[df2['q'],df2['jdg']]).to_csv('relation_first_check_'+str(fnum)+'.csv')

  #戦略ごとのチェック数
  df2 = df[df['jdg']=='s']
  # if index == 0:
  df2.to_csv('relation_st'+str(fnum)+'.csv')


  pd.pivot_table(df2,index='relation_st',columns=['q'],values='check',aggfunc=[np.mean,len]).to_csv('strategy_check_'+str(fnum)+'.csv')
  pd.pivot_table(df2,index='relation_st',columns=['q'],values='stp',aggfunc=[np.mean,len]).to_csv('strategy_step_'+str(fnum)+'.csv')